#                                          STONE PAPER SCISSOR GAME

# IMPORTING REQUIRED LIBRARIES

In [7]:
import tkinter
from tkinter import*
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

# WRITING TWO PLAYER FUNCTION

In [8]:
def TwoplayerGame():
    capture=cv2.VideoCapture(0)
    print(capture.get(cv2.CAP_PROP_FRAME_WIDTH))#Changing the size of the window
    print(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    ret = capture.set(cv2.CAP_PROP_FRAME_HEIGHT,1080)
    ret = capture.set(cv2.CAP_PROP_FRAME_WIDTH,1080)
    while capture.isOpened():
        ret,frame=capture.read()
        cv2.line(frame,(650,0),(650,900),(255,255,255),2)
        cv2.rectangle(frame,(100,100),(500,500),(255,255,255),2)#Drawing the first rectangle for player 1
        cv2.rectangle(frame,(800,100),(1200,500),(255,255,255),2)#Drawing the second rectangle for player two
        cv2.putText(frame, "PLAYER 1", (50, 600), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,255,255),2)
        cv2.putText(frame, "PLAYER 2", (700, 600), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,255,255),2)
        
        firstp=frame[100:500,100:500]#Getting the ROI for player 1
        secondp=frame[100:500,800:1200]#Getting the ROI for player 2
        #removing noise
        firstp=cv2.GaussianBlur(firstp,(5,5),0)#filtering the first player gesture
        secondp=cv2.GaussianBlur(secondp,(5,5),0)#filtering the second player gesture
        
        #Finding the hsv image
        hsv1=cv2.cvtColor(firstp,cv2.COLOR_BGR2HSV)#finding hsv for the first player gesture
        hsv2=cv2.cvtColor(secondp,cv2.COLOR_BGR2HSV)#finding hsv for the second player gesture
        lower_skin=np.array([0,30,60])
        upper_skin=np.array([20,150,255])
        #Create a binary image
        first_binary=cv2.inRange(hsv1,lower_skin,upper_skin)#binary image for first player gesture
        second_binary=cv2.inRange(hsv2,lower_skin,upper_skin)#binary image for first player gesture
    
        #morphological transformations
        kernel=np.ones((3,3),np.uint8)#kernel for morphological transformations
    
        #morphological transformation for player one gesture
        dilution1=cv2.dilate(first_binary,kernel,iterations=1)
        erosion1=cv2.erode(dilution1,kernel,iterations=1)
    
        #morphological transformation for player 2 gesture
        dilution2=cv2.dilate(second_binary,kernel,iterations=1)
        erosion2=cv2.erode(dilution2,kernel,iterations=1)
    
        #Again filtering the 
        ret1,the1=cv2.threshold(erosion1,70,255,cv2.THRESH_BINARY)
        #cv2.imshow('Thresholded_image1',the1)
        ret2,the2=cv2.threshold(erosion2,70,255,cv2.THRESH_BINARY)
        #cv2.imshow('Thresholded_image2',the2)
    
        #Drawing And Finding Contours and convex hull of the first player hand gesture
        contours1,heirchy=cv2.findContours(the1.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        try:
            contour = max(contours1, key=lambda x: cv2.contourArea(x))#Drawing the maximum size contour
            drawing1 = np.zeros(firstp.shape, np.uint8)#Drawing the window to display contour
            hull=cv2.convexHull(contour)
            
            cv2.drawContours(drawing1, [contour], -1, (0,255,0), 2)#drawing the contour on drawing window
            cv2.drawContours(drawing1, [hull], -1, (0,0,255), 2)#Drawing the hull on drawing window
            #Find convexity defects
            hull = cv2.convexHull(contour, returnPoints=False)
            defects = cv2.convexityDefects(contour, hull)
            # Use cosine rule to find angle of the far point from the start and end point i.e. the convex points (the finger
            # tips) for all defects
            count_defects = 0
            all_image1 = np.hstack((drawing1, firstp))
            cv2.imshow('Contour for player 1', all_image1)

            for i in range(defects.shape[0]):
                s, e, f, d = defects[i, 0]
                start = tuple(contour[s][0])
                end = tuple(contour[e][0])
                far = tuple(contour[f][0])
            
                #If the angle of hand is changed
                a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
                b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
                c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
                angle = (math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / 3.14

                #if angle > 90 draw a circle at the far point
                if angle <= 90:
                    count_defects += 1
                    cv2.circle(firstp, far, 1, [0, 0, 255], -1)
                    cv2.line(firstp, start, end, [0, 255, 0], 2)
                #Print the Gestures
            if count_defects == 0:
                cv2.putText(frame, "STONE", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255),2)
            elif count_defects == 1:
                cv2.putText(frame, "Scissor", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
            elif count_defects == 4:
                cv2.putText(frame, "Paper", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
            else:
                pass
    
        except:
            pass
        #Drawing And Finding Contours and convex hull of the second player hand gestur
        contours2,heirchy=cv2.findContours(the2.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        try:
            contour = max(contours2, key=lambda x: cv2.contourArea(x))#Drawing the maximum size contour
            drawing2 = np.zeros(secondp.shape, np.uint8)#Drawing the window to display contour
            hull=cv2.convexHull(contour)
            
            cv2.drawContours(drawing2, [contour], -1, (0,255,0), 2)#drawing the contour on drawing window
            cv2.drawContours(drawing2, [hull], -1, (0,0,255), 2)#Drawing the hull on drawing window
             #Find convexity defects
            hull = cv2.convexHull(contour, returnPoints=False)
            defects = cv2.convexityDefects(contour, hull)

            # Use cosine rule to find angle of the far point from the start and end point i.e. the convex points (the finger
            # tips) for all defects
            count_defects = 0
            all_image2 = np.hstack((drawing2, secondp))  #Horizontal stack-combines drawing2 and player 2 gesture
            cv2.imshow('Contours for player 2', all_image2)

            for i in range(defects.shape[0]):
                s, e, f, d = defects[i, 0]
                start = tuple(contour[s][0])
                end = tuple(contour[e][0])
                far = tuple(contour[f][0])
                #If the angle of hand is changed
                a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
                b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
                c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
                angle = (math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / 3.14
                
                #if angle > 90 draw a circle at the far point
                if angle <= 90:
                    count_defects += 1
                    cv2.circle(firstp, far, 1, [0, 0, 255], -1)
                    cv2.line(firstp, start, end, [0, 255, 0], 2)
            #Print the Gestures
            if count_defects == 0:
                cv2.putText(frame, "STONE", (655, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255),2)
            elif count_defects == 1:
                cv2.putText(frame, "Scissor", (655, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
            elif count_defects == 4:
                cv2.putText(frame, "Paper", (655, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
            else:
                pass
            
        except:
            pass
        
     #Show required images
        cv2.imshow("The STONE, PAPER & SCISSOR GAME", frame)
        #all_image1 = np.hstack((drawing1, firstp))  #Horizontal stack-combines drawing1 and player 1 gesture
        #cv2.imshow('Contour for player 1', all_image1)
        #all_image2 = np.hstack((drawing2, secondp))  #Horizontal stack-combines drawing2 and player 2 gesture
        #cv2.imshow('Contours for player 2', all_image2)
        if cv2.waitKey(1)==ord('q'):
            break
    capture.release()
    cv2.destroyAllWindows()

# CREATING GUI FOR GAME

In [9]:
window=tkinter.Tk()#creating the application window
window.title("GAME STARTED")
window.geometry('200x150')
single_player=tkinter.Button(window,text="Two Player",fg="black",bg="orange",command=TwoplayerGame)#Creating button
single_player.pack(side = TOP, pady = 5)#Setting the position of the button
Multi_player=tkinter.Button(window,text="Single Player",fg="black",bg="yellow").pack()#Creating Button
window.mainloop()

640.0
480.0
